In [6]:
import pandas as pd

Answer to exercises from notebook 2:



`wine.groupby('country')['description'].count().sort_values(ascending=False).plot(kind='bar', figsize=(10,7))
plt.show()`

# Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [7]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [8]:
print('Does the wine df conatin nulls?:', wine.isnull().any().any()) # first any looks at the rows, second at the columns
print('Does the energy df contain nulls?:', energy.isnull().any())

Does the wine df conatin nulls?: True
Does the energy df contain nulls?: PJM_Load_MW    False
dtype: bool


**Question:** why did we use ".any()" twice when asking this question for the wine analysis?

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

In [9]:
wine.isnull().any()

country         True
description    False
designation     True
points         False
price           True
province        True
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

In [11]:
print('Shape of wine table before is: ', wine.shape)

null_country = wine[wine['country'].isnull()]
print('Shape of null countries table is:', null_country.shape)

wine = wine[~wine.index.isin(null_country.index)] # ~ (tilde) means 'NOT'
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150930, 10)
Shape of null countries table is: (5, 10)
Shape of wine table after is:  (150925, 10)


The 5 rows where a NaN value existed for country have been dropped.

Now we'll do the same for rows with a null price.

In [12]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine[wine['price'].isnull()]
print('Shape of null prices table is:', null_price.shape)

#wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150925, 10)
Shape of null prices table is: (13695, 10)
Shape of wine table after is:  (150925, 10)


Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

In [13]:
wine['price'].fillna(wine['price'].mean(), inplace=True)

Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

In [14]:
wine.isnull().any()

country        False
description    False
designation     True
points         False
price          False
province       False
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

In [15]:
wine.fillna('unknown', inplace=True)

We now have no null values in our dataset!

In [16]:
wine.isnull().any()

country        False
description    False
designation    False
points         False
price          False
province       False
region_1       False
region_2       False
variety        False
winery         False
dtype: bool

In [17]:
wine

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.000000,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.000000,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.000000,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.000000,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.000000,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude
5,Spain,"Deep, dense and pure from the opening bell, th...",Numanthia,95,73.000000,Northern Spain,Toro,unknown,Tinta de Toro,Numanthia
6,Spain,Slightly gritty black-fruit aromas include a s...,San Román,95,65.000000,Northern Spain,Toro,unknown,Tinta de Toro,Maurodos
7,Spain,Lush cedary black-fruit aromas are luxe and of...,Carodorum Único Crianza,95,110.000000,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez
8,US,This re-named vineyard was formerly bottled as...,Silice,95,65.000000,Oregon,Chehalem Mountains,Willamette Valley,Pinot Noir,Bergström
9,US,The producer sources from two blocks of the vi...,Gap's Crown Vineyard,95,60.000000,California,Sonoma Coast,Sonoma,Pinot Noir,Blue Farm


### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

In [18]:
wine.loc[:, 'Unique ID'] =  wine.loc[:, 'country'] + "_" + wine.loc[:, 'winery']
# wine['Unique ID'] = wine.loc[:, 'country'] + "_" + wine.loc[:, 'winery']

#### Using a function + apply to make a new column:
Lets say we want to make a new column for "great, cheap wine".

We will define this as any wine that costs less than $12 and has a rating better than 95.

In [22]:
def helper(row):
    if (row['price'] < 12) & (row['points'] >=80):
        x = 1
    else:
        x = 0
    return x

In [23]:
wine['great and cheap'] = wine.apply(helper, axis=1)

There are NO great and cheap wines?!

In [24]:
wine[wine['great and cheap']==1]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Unique ID,great and cheap
90,US,"Bright, light oak shadings dress up this mediu...",unknown,86,10.0,California,California,California Other,Cabernet Sauvignon,Belle Ambiance,US_Belle Ambiance,1
94,Portugal,This family-owned estate has produced an immed...,Muros de Vinha,86,10.0,Douro,unknown,unknown,Portuguese Red,Quinta do Portal,Portugal_Quinta do Portal,1
99,France,This is a blend of 60% Ugni Blanc and 40% Colo...,Pigmentum,86,10.0,Southwest France,Côtes de Gascogne,unknown,Ugni Blanc-Colombard,Georges Vigouroux,France_Georges Vigouroux,1
107,US,This medium-bodied wine has more earthy and fl...,unknown,86,10.0,California,California,California Other,Chardonnay,Leaping Horse,US_Leaping Horse,1
119,Portugal,"This broad, fruity wine brings out ripe berry ...",Convento da Vila,86,7.0,Alentejano,unknown,unknown,Portuguese Red,Adega Cooperativa de Borba,Portugal_Adega Cooperativa de Borba,1
196,US,"A creamy, buttery texture wraps around a core ...",unknown,87,10.0,California,California,California Other,Chardonnay,Grand Pacific,US_Grand Pacific,1
204,US,This is an aromatic wine with notes of rose wa...,unknown,87,10.0,Washington,Columbia Valley (WA),Columbia Valley,Gewürztraminer,Chateau Ste. Michelle,US_Chateau Ste. Michelle,1
205,US,"The aromas of this wine suggest plum, herb and...",unknown,87,11.0,Washington,Columbia Valley (WA),Columbia Valley,Merlot,Castle Rock,US_Castle Rock,1
247,Romania,"This playful pink has fun aromas of cinnamon, ...",La Umbra,88,9.0,Colinele Dobrogei,unknown,unknown,Rosé,Cramele Halewood,Romania_Cramele Halewood,1
454,Portugal,This wine is full of red berry flavor and soft...,Conde Vimioso Colheita Seleccionada,86,10.0,Tejo,unknown,unknown,Portuguese Red,Falua,Portugal_Falua,1


**Exercise:** Using a function, make a column that indicates if a wine comes from Africa.

**Exercise:** Make a new column in the energy dataset, showing the energy in time T-1 for each time T. Hint: there is a built-in method in Pandas that can help with this, no complicated functions needed :)

**Exercise:** Re-read the CSV for the wine data, and write a function to take care of all the nan values. Bonus points if it includes a test.

In [29]:
wine.country.unique()

array(['US', 'Spain', 'France', 'Italy', 'New Zealand', 'Bulgaria',
       'Argentina', 'Australia', 'Portugal', 'Israel', 'South Africa',
       'Greece', 'Chile', 'Morocco', 'Romania', 'Germany', 'Canada',
       'Moldova', 'Hungary', 'Austria', 'Croatia', 'Slovenia', 'India',
       'Turkey', 'Macedonia', 'Lebanon', 'Serbia', 'Uruguay',
       'Switzerland', 'Albania', 'Bosnia and Herzegovina', 'Brazil',
       'Cyprus', 'Lithuania', 'Japan', 'China', 'South Korea', 'Ukraine',
       'England', 'Mexico', 'Georgia', 'Montenegro', 'Luxembourg',
       'Slovakia', 'Czech Republic', 'Egypt', 'Tunisia', 'US-France'],
      dtype=object)

In [30]:
african_countries = ['South Africa', 'Morocco', 'Egypt', 'Tunisia']

In [31]:
def in_africa(row):
    if row['country'] in african_countries:
        return 1
    else: 
        return 0

In [32]:
wine['african'] = wine.apply(in_africa, axis=1)

In [39]:
wine.loc[wine['african'] == 1]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Unique ID,great and cheap,african
114,South Africa,Muted aromas of tart gooseberry and lime peel ...,Circumstance,86,20.0,Stellenbosch,unknown,unknown,Sauvignon Blanc,Waterkloof,South Africa_Waterkloof,0,1
216,Morocco,"Tones of black cherry, toffee, tobacco, tar an...",Domaine Excelcio,93,35.0,Guerrouane,unknown,unknown,Syrah-Grenache,Bernard Magrez,Morocco_Bernard Magrez,0,1
461,South Africa,"Muted, soft scents of green apple, orange peel...",unknown,86,20.0,Walker Bay,unknown,unknown,Sauvignon Blanc,Bouchard Finlayson,South Africa_Bouchard Finlayson,0,1
721,South Africa,"This wine opens softly, with faint aromas of l...",Schaapenberg Vineyard Reserve,86,35.0,Stellenbosch,unknown,unknown,Sauvignon Blanc,Vergelegen,South Africa_Vergelegen,0,1
724,South Africa,"This opens with scents of green melon rind, st...",Barrelman's Select,86,11.0,Western Cape,unknown,unknown,Sauvignon Blanc,Fat Barrel,South Africa_Fat Barrel,1,1
734,South Africa,"Fresh grass, gooseberry, lime leaf and a touch...",unknown,85,10.0,Western Cape,unknown,unknown,Sauvignon Blanc,Long Neck,South Africa_Long Neck,1,1
741,South Africa,"This is soft and accessible now, with plush no...",Crocodile's Lair Kaaimansgat,85,20.0,Overberg,unknown,unknown,Chardonnay,Bouchard Finlayson,South Africa_Bouchard Finlayson,0,1
744,South Africa,"This is a fresh and lively white, with some gr...",The Game Reserve,85,16.0,Robertson,unknown,unknown,Sauvignon Blanc,Graham Beck,South Africa_Graham Beck,0,1
1006,South Africa,"Surprisingly shy and reserved in nature, this ...",unknown,85,22.0,Stellenbosch,unknown,unknown,Chardonnay,Vergelegen,South Africa_Vergelegen,0,1
1008,South Africa,"Bright aromas of tart apple, melon rind and fr...",unknown,85,10.0,Western Cape,unknown,unknown,Chenin Blanc,Essay,South Africa_Essay,1,1


In [42]:
energy['energy_previous'] = energy.shift(1)
energy

,PJM_Load_MW,energy_previous
Datetime,,
1998-12-31 01:00:00,29309.0,NaN
1998-12-31 02:00:00,28236.0,29309.0
1998-12-31 03:00:00,27692.0,28236.0
1998-12-31 04:00:00,27596.0,27692.0
1998-12-31 05:00:00,27888.0,27596.0
1998-12-31 06:00:00,29382.0,27888.0
1998-12-31 07:00:00,31373.0,29382.0
1998-12-31 08:00:00,33272.0,31373.0
1998-12-31 09:00:00,34133.0,33272.0


In [51]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
print('Does the wine df contain nulls?:', wine.isnull().any().any())

Does the wine df contain nulls?: True


In [52]:
wine.isnull().any()

country         True
description    False
designation     True
points         False
price           True
province        True
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

In [59]:
print(wine.shape)
for column in wine.columns:
    null_indices = wine[wine[column].isnull()]
    wine = wine[~wine.index.isin(null_indices.index)]
wine.shape

(150930, 10)


(39241, 10)

In [60]:
wine.isnull().any()

country        False
description    False
designation    False
points         False
price          False
province       False
region_1       False
region_2       False
variety        False
winery         False
dtype: bool